# Libraries

In [1]:
from _ctypes import LoadLibrary as _dlopen
_dlopen("D:\\Sangwons_Room\\00_SoftWares\\Anaconda\\envs\\torch\\Library\\bin\\geos.dll", 0)

# nuscenes-devkit
from nuscenes.nuscenes import NuScenes

# nuscenes-map expansion
from utils.custom_map_api_expansion import CustomNuScenesMap

# nuscenes-devkit-utils
from nuscenes.utils.data_classes import LidarPointCloud, RadarPointCloud, Box
from nuscenes.utils.geometry_utils import view_points, transform_matrix

from pyquaternion import Quaternion


from functools import reduce
import os

import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import tqdm

from torch.utils.data import Dataset, DataLoader
%matplotlib inline

# Set  Parameters

In [2]:
locations = ['singapore-onenorth', 'singapore-hollandvillage', 'singapore-queenstown', 'boston-seaport']
version = 'v1.0-mini'
dataroot = './data/sets/nuscenes'

# 

In [3]:
nusc = NuScenes(version=version, dataroot=dataroot, verbose=True)
nusc_location = dict()
for loc in locations:
    nusc_location[loc] = CustomNuScenesMap(dataroot=dataroot, map_name=loc)

Loading NuScenes tables for version v1.0-mini...
Loading nuScenes-lidarseg...
32 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
404 lidarseg,
Done loading in 0.679 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [4]:
scene = nusc.scene[0]
nusc.list_scenes()

scene-0061, Parked truck, construction, intersectio... [18-07-24 03:28:47]   19s, singapore-onenorth, #anns:4622
scene-0103, Many peds right, wait for turning car, ... [18-08-01 19:26:43]   19s, boston-seaport, #anns:2046
scene-0655, Parking lot, parked cars, jaywalker, be... [18-08-27 15:51:32]   20s, boston-seaport, #anns:2332
scene-0553, Wait at intersection, bicycle, large tr... [18-08-28 20:48:16]   20s, boston-seaport, #anns:1950
scene-0757, Arrive at busy intersection, bus, wait ... [18-08-30 19:25:08]   20s, boston-seaport, #anns:592
scene-0796, Scooter, peds on sidewalk, bus, cars, t... [18-10-02 02:52:24]   20s, singapore-queensto, #anns:708
scene-0916, Parking lot, bicycle rack, parked bicyc... [18-10-08 07:37:13]   20s, singapore-queensto, #anns:2387
scene-1077, Night, big street, bus stop, high speed... [18-11-21 11:39:27]   20s, singapore-hollandv, #anns:890
scene-1094, Night, after rain, many peds, PMD, ped ... [18-11-21 11:47:27]   19s, singapore-hollandv, #anns:1762
sc

In [5]:
import sys
for var in dir():
    print(var, sys.getsizeof(sys))

Box 80
CustomNuScenesMap 80
DataLoader 80
Dataset 80
In 80
LidarPointCloud 80
NuScenes 80
Out 80
Quaternion 80
RadarPointCloud 80
_ 80
__ 80
___ 80
__builtin__ 80
__builtins__ 80
__doc__ 80
__loader__ 80
__name__ 80
__package__ 80
__spec__ 80
_dh 80
_dlopen 80
_i 80
_i1 80
_i2 80
_i3 80
_i4 80
_i5 80
_ih 80
_ii 80
_iii 80
_oh 80
dataroot 80
exit 80
get_ipython 80
loc 80
locations 80
np 80
nusc 80
nusc_location 80
os 80
plt 80
px 80
quit 80
reduce 80
scene 80
sys 80
tqdm 80
transform_matrix 80
version 80
view_points 80


# DataLoader (TBD)

In [6]:
# locations = ['singapore-onenorth', 'singapore-hollandvillage', 'singapore-queenstown', 'boston-seaport']

# class NuscensDataLoader(Dataset):
#     def __init__(self, version:str = 'v1.0-mini', dataroot:str = './data/sets/nuscenes', data_type = 'Train'):
#         self.data_type = data_type
#         self.nusc = NuScenes(version=version, dataroot=dataroot, verbose=True)
#         self.nusc_location = dict()
        
#         for loc in locations:
#             nusc_location[loc] = CustomNuScenesMap(dataroot=dataroot, map_name=loc)
    
#     def __len__(self):
#         return 0
    
#     def __getindex__(self,idx):
#         return None, None
    
    
# class DataSpliter():
#     def __init__(self, version:str = 'v1.0-mini', dataroot:str = './data/sets/nuscenes', data_type = 'Train')
    
# NuscensDataLoader()

# 

In [1]:
my_scene = nusc.scene[0]
first_sample_token = my_scene['first_sample_token']
last_sample_token = my_scene['last_sample_token']

sample_rec = nusc.get('sample', last_sample_token)
last_time = sample_rec['timestamp']

sample_rec = nusc.get('sample', first_sample_token)
first_time = sample_rec['timestamp']

ref_sd_token = sample_rec['data']['LIDAR_TOP']
ref_sd_rec = nusc.get('sample_data', ref_sd_token)
ref_pose_rec = nusc.get('ego_pose', ref_sd_rec['ego_pose_token'])
ref_cs_rec = nusc.get('calibrated_sensor', ref_sd_rec['calibrated_sensor_token'])
ref_time = 1e-6 * ref_sd_rec['timestamp']

# Homogeneous transform from ego car frame to reference frame.
ref_from_car = transform_matrix(ref_cs_rec['translation'], Quaternion(ref_cs_rec['rotation']), inverse=True)

# Homogeneous transformation matrix from global to _current_ ego car frame.
car_from_global = transform_matrix(ref_pose_rec['translation'], Quaternion(ref_pose_rec['rotation']),
                                   inverse=True)

sample_data_token = sample_rec['data']['LIDAR_TOP']
current_sd_rec = nusc.get('sample_data', sample_data_token)

print(last_time - first_time)
for i in range(1000):
    print(f"{i} : {current_sd_rec['timestamp'] - first_time}")
    current_pc = LidarPointCloud.from_file(os.path.join(nusc.dataroot, current_sd_rec['filename']))

    current_pose_rec = nusc.get("ego_pose", current_sd_rec['ego_pose_token'])
    global_from_car = transform_matrix(current_pose_rec['translation'], Quaternion(current_pose_rec['rotation']), inverse = False)

    current_cs_rec = nusc.get('calibrated_sensor', current_sd_rec['calibrated_sensor_token'])
    car_from_current = transform_matrix(current_cs_rec['translation'], Quaternion(current_cs_rec['rotation']), inverse =False)

    trans_matrix = reduce(np.dot, [ref_from_car, car_from_global, global_from_car, car_from_current])
    current_pc.transform(trans_matrix)

    if current_sd_rec['next'] == "":
        break
    else:
        current_sd_rec = nusc.get('sample_data', current_sd_rec['next'])

NameError: name 'nusc' is not defined

In [8]:
ref_pose_rec

{'token': '9d9bf11fb0e144c8b446d54a8a00184f',
 'timestamp': 1532402927647951,
 'rotation': [0.5720320396729045,
  -0.0016977771610471074,
  0.011798001930183783,
  -0.8201446642457809],
 'translation': [411.3039349319818, 1180.8903791765097, 0.0]}

In [9]:
transform_matrix(ref_pose_rec['translation'], Quaternion(ref_pose_rec['rotation']),
                                   inverse=True)

array([[-3.45552926e-01, -9.38338111e-01, -1.07128245e-02,
         1.25020173e+03],
       [ 9.38257989e-01, -3.45280305e-01, -2.12945025e-02,
         2.18289878e+01],
       [ 1.62825160e-02, -1.74097708e-02,  9.99715849e-01,
         1.38619679e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [ ]:
transform_matrix(ref_pose_rec['translation'], Quaternion(ref_pose_rec['rotation']),
                                   inverse=True)
